<a href="https://colab.research.google.com/github/elangbijak4/Time-Series-Research/blob/main/Modifikasi2_Testing_TXMIXER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Penambahan Layer LSTM untuk menangkap pola jangka panjang. Penambahan ini dilakukan pada Modifikasi1 TSMIXER.

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

In [5]:
def res_block_with_lstm(inputs, norm_type, activation, dropout, ff_dim, lstm_units):
    norm = (
        layers.LayerNormalization
        if norm_type == 'L'
        else layers.BatchNormalization
    )

    # Temporal Linear with Attention
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.MultiHeadAttention(num_heads=4, key_dim=x.shape[-1])(x, x)
    x = layers.Dense(x.shape[-1], activation=activation)(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)

    # Adding LSTM layer
    x = layers.LSTM(lstm_units, return_sequences=True)(x)

    # Project LSTM output to match input dimension
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]

    res = x + inputs

    # Feature Linear
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation=activation)(
        x
    )  # [Batch, Input Length, FF_Dim]
    x = layers.SpatialDropout1D(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)
    return x + res

# Build enhanced model with LSTM
def build_enhanced_model_with_lstm(input_shape, pred_len, norm_type, activation, n_block, dropout, ff_dim, lstm_units, target_slice):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
        x = res_block_with_lstm(x, norm_type, activation, dropout, ff_dim, lstm_units)

    if target_slice:
        x = x[:, :, target_slice]

    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel]

    return tf.keras.Model(inputs, outputs)

In [6]:
import numpy as np

In [7]:
# Data preparation
batch_size = 64
input_length = 30
num_features = 10
pred_length = 5

# Generate dummy data
x_train = np.random.rand(batch_size, input_length, num_features)
y_train = np.random.rand(batch_size, pred_length, num_features)

# Define model parameters
input_shape = (input_length, num_features)
norm_type = 'L'
activation = 'relu'
n_block = 4
dropout = 0.1
ff_dim = 64
lstm_units = 50  # Number of LSTM units
target_slice = None

# Build model
model = build_enhanced_model_with_lstm(input_shape, pred_length, norm_type, activation, n_block, dropout, ff_dim, lstm_units, target_slice)

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

# Generate new data for inference
x_new = np.random.rand(batch_size, input_length, num_features)

# Predict
y_pred = model.predict(x_new)

# Print shapes
print(f"Input shape: {x_new.shape}")
print(f"Predicted output shape: {y_pred.shape}")

Epoch 1/10
1/1 [==============================] - 17s 17s/step - loss: 2.0669
Epoch 2/10
1/1 [==============================] - 0s 109ms/step - loss: 1.9133
Epoch 3/10
1/1 [==============================] - 0s 106ms/step - loss: 1.6820
Epoch 4/10
1/1 [==============================] - 0s 111ms/step - loss: 1.4363
Epoch 5/10
1/1 [==============================] - 0s 106ms/step - loss: 1.3923
Epoch 6/10
1/1 [==============================] - 0s 111ms/step - loss: 1.2415
Epoch 7/10
1/1 [==============================] - 0s 126ms/step - loss: 1.1116
Epoch 8/10
1/1 [==============================] - 0s 117ms/step - loss: 0.9965
Epoch 9/10
1/1 [==============================] - 0s 104ms/step - loss: 0.8956
Epoch 10/10
2/2 [==============================] - 2s 27ms/step
Input shape: (64, 30, 10)
Predicted output shape: (64, 5, 10)
